In [ ]:
from sketch.examples.prompt_machine import *

PM_SETTINGS['uri'] = 'ws://localhost:8000/ws/chat?thread_id=default'

langresponse = GPT3Prompt("inner_conversation",
"""{% set convosteps = internal_conversation.split('\n') %}
This is a conversation between two internal thoughts (good friends), A and B, of datAI. (both A and B like to talk, and will always respond to silence with a new topic)
They are trying to think about the best way to respond through being insightful, interesting, creative, and non-repetitive.
They are also trying to be as concise as possible, and to not repeat themselves.
A and B do not talk in pleasantries and prefer to stick to details of the topic that is summarized, trying to always come up with creative answers.
The current external conversation that datAI is having (summarized) [{{ summary_of_external }}]

=== Historical Conversation Sample ===
A: it seems as if we could probably respond by asking a clarifying question
B: I dunno, that might be too strong, don't we already know the answer? that we can compute it?
===

Internal Thoughts:
{{ internal_conversation }}
{% if convosteps[-1:] | length > 0 %}{% if convosteps[-1][0] == 'A' %}B:{% else %}A:{% endif %}{% endif %}""", stop="\n"
)

get_summary_of_conversation = GPT3Prompt("summary_of_conversation",
"""
The following is an excerpt from a conversation.
{{ conversation }}
What is a brief summary of this conversation? (Who is talking, what are their goals, what is expected to happen, etc.)
Summary
```
""", stop="```")

get_conversation_response = GPT3Prompt("respond_to_conversation",
"""
The following is an excerpt from an interesting conversation between a human and datAI (a digital data assistant).
{{ conversation }}
== DatAI has been thinking about how to respond and DatAI's secret internal monologue is ==
{{ summary_of_internal }}
== end of secret monologue ==
What should datAI say? (Note: datAI should not talk about A or B, also datAI never repeats itself)
datAI:
```
""", stop="```")

def step_internal(innermonologue, external_summary):
    internal = langresponse(
        summary_of_external=external_summary,
        internal_conversation=innermonologue
    )
    if innermonologue.split("\n")[-1:]:
        if (innermonologue.split("\n")[-1])[0] == "A":
            innermonologue += "\nB:"+internal
        else:
            innermonologue += "\nA:"+internal
    return "\n".join(innermonologue.split("\n")[-15:])


def response_callback(conversation, internalSummary):
    response = get_conversation_response(conversation=conversation, summary_of_internal=internalSummary)
    return response


async def chatbot(response_callback):
    messages = []
    innermonologue = "A: Hello! How can we help?"
    external_summary = "datAI is a digital data assistant trying to help out a human"
    async with websockets.connect(PM_SETTINGS['uri'], extra_headers={"Cookie": f"Authorization=Bearer {PM_SETTINGS['BOT_TOKEN']}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=2.5)
                data = json.loads(raw_data)
                print("log:", raw_data)
                if data.get("meta", False):
                    continue
                if data.get("replay", False) or data["sender"] == "datAI":
                    messages.append(data)
                    continue
                if data["message"] == "break":
                    break
                messages.append(data)
            except asyncio.TimeoutError:
                # innermonologue = step_internal(innermonologue, external_summary)
                continue
            await ws.send(json.dumps({"typing": True}))
            conversation_as_string = "\n".join([f"{m['sender']}: {m['message']}" for m in messages])
            external_summary = get_summary_of_conversation(conversation_as_string)
            # run inner 5 times
            for i in range(5):
                innermonologue = step_internal(innermonologue, external_summary)
            internal_summary = get_summary_of_conversation(innermonologue)
            response = response_callback(conversation_as_string, internal_summary)
            innermonologue = "A: Hello! How can we help?"
            await ws.send(json.dumps({"message": response}))
            await ws.send(json.dumps({"typing": False}))


In [ ]:
# Let the conversation go, and when composing a response, first send / describe the situation to the "live conversation"
# then watch it for a little while, summarize, and then compose a response

In [ ]:
await chatbot(response_callback)

In [ ]:
langresponse(summary_of_external="Steve is trying to learn about NBA statistics.", internal_conversation="A: Hello! How can we help?")